## 라이브러리 가져오기

In [ ]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time

import pandas as pd

import re

## 동아일보 사건사고

In [ ]:
# 데이터프레임 생성
accident_df = pd.DataFrame(columns=['date', 'title', 'body', 'URL'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
options.add_argument('headless') # headless 모드 설정 (창 안 띄움)
options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')

driver = webdriver.Chrome(chrome_path, options=options)

# 맨 처음 검색할 날짜 지정
date = "20191128"

# 수집한 기사 개수 카운트
article_cnt = 0

while (True):
    # page 수 계산한 다음 그만큼 반복
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        if p > 1:
            url = "https://www.donga.com/news/Society/Event?p={}1&prod=news&ymd={}&m=".format((p-1)*2, date)
        else:
            url = "https://www.donga.com/news/Society/Event?ymd={}&m=".format(date)
            
        driver.get(url)
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("page")) - 2
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.articleList"))
   
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len == 1: # 해당 날짜에 기사가 없는 경우
                break;
           
            i = i+3
            a = driver.find_element_by_css_selector("#content > div:nth-child({}) > div.rightList > a".format(i))
            
            # url 수집
            link = a.get_attribute('href')
            
            # 제목 수집
            t = driver.find_element_by_css_selector("#content > div:nth-child({}) > div.rightList > a > span.tit".format(i))
            tit = t.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = driver.find_element_by_css_selector("#content > div > div.article_txt").text
            
            # 데이터 프레임에 삽입
            accident_df.loc[article_cnt] = [date, tit, body, link]
            article_cnt += 1
            
            print(date)
            print(tit)
            print(link)
            print("------------------------------------------------")
                
            # 확인을 위한 break
            if article_cnt==3000:
                break
            
            i = i - 3
            driver.back()
            driver.implicitly_wait(1)
            

        p += 1
        if article_cnt==3000:
                break

    if article_cnt==3000:
                break
            
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,1).until(EC.element_to_be_clickable((By.CLASS_NAME, "prev.on")))
    btn.send_keys(Keys.ENTER)
    date = re.sub(r'[^0-9]', '', WebDriverWait(driver,1).until(lambda x : x.find_element_by_css_selector("#content > div.dateDivide > span.dateBox > span")).text)

driver.close()

In [31]:
accident_df

,date,title,body,URL
0,20191128,경찰 “靑에 ‘김기현 수사’ 9번 정보공유…별지시 없어”,"""언론 보도, 정당 이의 등 상황 정보 공유""\n""압수수색 당일 알아""…사전보고 의...",https://www.donga.com/news/List/article/all/20...
1,20191128,경찰 “조국 민정실 파견경찰이 김기현 밀봉첩보 들고왔다”,황운하 대전지방경찰청장이 27일 오후 대전지방경찰청 브리핑실에서 긴급 기자회견을 열...,https://www.donga.com/news/List/article/all/20...
2,20191128,급식직원들 싸움에 1600명 초등생 점심 굶어…학부모들 ‘분통’,지난 26일 급식중단 사태가 발생한 경기 시흥시의 한 초등학교 학부모가 학교로부터 ...,https://www.donga.com/news/List/article/all/20...
3,20191128,작년 9명사상 한화 폭발사고 공장장 등 금고 1년~1년6월 구형,지난해 5월29일 대전 유성구 외삼동 한화 대전공장에 폭발사고 후 소방대가 출동하는...,https://www.donga.com/news/List/article/all/20...
4,20191128,‘위험한 질주’…고속버스 기사 운전 중 동영상 시청,전북의 한 고속버스 운전기사가 운전중 동영상을 시청하고 있다.(제보자 제공) 뉴스1...,https://www.donga.com/news/List/article/all/20...
...,...,...,...,...
1702,20190818,‘프로포폴 투약 위해 18번 내시경’…30대 검찰 송치,"3차례 위내시경 받고 20만원 안 내기도\n6개월간 190㎖가량 투약,정신병원 입원...",https://www.donga.com/news/List/article/all/20...
1703,20190818,‘영장심사’ 한강 토막살인 피의자 “다음에 또 그러면 너 또 죽는다” 막말,‘한강 몸통 시신’ 사건의 피의자 A(39·모텔 종업원)씨가 18일 오후 경기 의정...,https://www.donga.com/news/List/article/all/20...
1704,20190818,‘한강 몸통 시신’ 피의자 영장심사 출석…“피해자가 먼저 시비” 당당,사진=뉴스1\n자신을 기분 나쁘게 했다는 이유로 손님을 잔혹하게 살해한 뒤 사체를 ...,https://www.donga.com/news/List/article/all/20...
1705,20190818,“기분 나빠 홧김에”…‘한강 몸통 시신’ 피의자 18일 영장심사,사진=뉴시스\n자신을 기분 나쁘게 했다는 이유로 손님을 잔혹하게 살해한 뒤 사체를 ...,https://www.donga.com/news/List/article/all/20...
